In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer
import numpy as np

In [5]:
df = pd.read_csv("projeto de sistema de recomendação.csv")

In [6]:
df.head(8)

,user_id,product_id,rating
0,1,101,5
1,1,102,4
2,2,101,4
3,2,103,5
4,3,102,3
5,3,103,4
6,4,101,5
7,4,104,5


In [7]:
user_item_matrix = df.pivot_table(index='user_id', columns='product_id', values='rating')
user_item_matrix.head()

product_id,101,102,103,104
user_id,,,,
1,5.0,4.0,NaN,NaN
2,4.0,NaN,5.0,NaN
3,NaN,3.0,4.0,NaN
4,5.0,NaN,NaN,5.0
5,NaN,NaN,4.0,NaN


In [8]:
imputer = SimpleImputer(strategy='mean')
user_item_matrix_filled = imputer.fit_transform(user_item_matrix)



In [9]:
user_similarity = cosine_similarity(user_item_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [10]:
def recommend_products(user_id, user_item_matrix, similarity_matrix, top_n=3):
    user_idx = user_id - 1
    similar_users = similarity_matrix[user_idx]
    
    
    scores = np.dot(similar_users, user_item_matrix_filled)
    
    
    user_rated = user_item_matrix.loc[user_id].dropna().index
    
    
    recommendations = pd.Series(scores, index=user_item_matrix.columns).drop(user_rated, errors="ignore")
    
    
    top_recommendations = recommendations.sort_values(ascending=False).head(top_n)
    return top_recommendations.index.tolist()



In [11]:
user_id = 1
recommended_products = recommend_products(user_id, user_item_matrix, user_similarity_df.values)
print(f"Produtos recomendados para o usuário {user_id}: {recommended_products}")


Produtos recomendados para o usuário 1: [104, 103]
